In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

In [5]:
%run othello_game.ipynb
%run othello_ai.ipynb

#### Ermitteln der Standardabweichung
Im Folgenden werden zunächst einige Datenpunkte gesammelt, indem in verschiedenen Spielzuständen jeweils eine tiefe und eine flache Suche durchgeführt wird. Die Spielzustände werden durch zufälliges Ziehen erreicht. Die Daten werden in einer CSV-Datei gespeichert.

In [ ]:
import csv
import os


def sample_probcut_values(num_games, shallow_depth, deep_depth):
    fname = f'probcut_dataset_{PROBCUT_SHALLOW_DEPTH}_{PROBCUT_DEEP_DEPTH}.csv'
    file_exists = os.path.isfile(fname)
    if file_exists:
        print('using existing dataset')
    else:
        with open(fname, 'w', newline='') as file:
            writer = csv.writer(file, delimiter=',')
            writer.writerow(('moves', 'shallow', 'deep'))
            for i in range(num_games):
                state = GameState()
                while not state.game_over:
                    state = ai_make_move(random_ai, state, 0, None)
                    shallow_value = alphabeta(
                        state, PROBCUT_SHALLOW_DEPTH,
                        combined_heuristic, -math.inf, math.inf
                    )
                    deep_value = alphabeta(
                        state, PROBCUT_DEEP_DEPTH,
                        combined_heuristic, -math.inf, math.inf
                    )
                    print(f'shallow: {shallow_value}, deep: {deep_value}')
                    writer.writerow(
                        (state.num_pieces, shallow_value, deep_value))
            file.close()

In [ ]:
sample_probcut_values(100, PROBCUT_SHALLOW_DEPTH, PROBCUT_DEEP_DEPTH)

Laden der Daten aus der CSV-Datei mithilfe von Pandas

In [6]:
import pandas
filename = f'probcut_dataset_{PROBCUT_SHALLOW_DEPTH}_{PROBCUT_DEEP_DEPTH}.csv'
df = pandas.read_csv(filename)
shallow = np.array(df['shallow'])
deep = np.array(df['deep'])
moves = np.array(df['moves'])

Daten Visualisieren

In [ ]:
import matplotlib.pyplot as plt
import sklearn.linear_model as lm
import seaborn as sns
model = lm.LinearRegression()
model.fit(shallow.reshape(len(shallow), 1), deep)
plt.figure(figsize=(15, 10))
sns.set(style='whitegrid')
plt.scatter(shallow, deep)
plt.axvline(x=0.0, c='k')
plt.axhline(y=0.0, c='k')
plt.plot(shallow, shallow * model.coef_ + model.intercept_)
plt.xlabel('shallow search')
plt.ylabel('deep search')
plt.title('Probcut Values')
plt.show()

Folgender Code berechnet die Standardabweichung pro Anzahl Steine auf dem Spielfeld. Dazu werden zunächst für jede Anzahl an Steinen aus den Daten die passenden Werte extrahiert. Für diese Teilmengen wird jeweils die Varianz mit $numpy$ berechnet. Die Standardabweichung ist die positive Wurzel aus der Varianz. Die Anzahl an Steinen und die Standardabweichung werden an entsprechende Felder angehängt.

In [10]:
import numpy as np
import sklearn.metrics as skl
import sklearn.linear_model as lm
import matplotlib.pyplot as plt
import seaborn as sns
import math

x = []
y = []

for i in range(5, 45):
    shallow_c = shallow[moves == i]
    deep_c = deep[moves == i]
    variance = np.var(np.stack([shallow_c, deep_c], axis=1))
    explained_variance = skl.explained_variance_score(shallow_c, deep_c)
    x.append(i)
    y.append(math.sqrt(variance))
    #print(f'{i}: Var={round(variance, 5)}, explained: {round(explained_variance, 3)}')

5: Var=0.00019, explained: 0.938
6: Var=0.00039, explained: 0.786
7: Var=0.00154, explained: 0.932
8: Var=0.0009, explained: 0.693
9: Var=0.0021, explained: 0.877
10: Var=0.00206, explained: 0.864
11: Var=0.00296, explained: 0.924
12: Var=0.00352, explained: 0.936
13: Var=0.0034, explained: 0.948
14: Var=0.00436, explained: 0.953
15: Var=0.00478, explained: 0.959
16: Var=0.00424, explained: 0.961
17: Var=0.00519, explained: 0.955
18: Var=0.00515, explained: 0.967
19: Var=0.00545, explained: 0.957
20: Var=0.00547, explained: 0.969
21: Var=0.00569, explained: 0.967
22: Var=0.00601, explained: 0.968
23: Var=0.00549, explained: 0.949
24: Var=0.00601, explained: 0.96
25: Var=0.00643, explained: 0.96
26: Var=0.00679, explained: 0.961
27: Var=0.00693, explained: 0.968
28: Var=0.00661, explained: 0.964
29: Var=0.00653, explained: 0.964
30: Var=0.00735, explained: 0.967
31: Var=0.0075, explained: 0.954
32: Var=0.00847, explained: 0.959
33: Var=0.00733, explained: 0.943
34: Var=0.00769, explaine

Folgender Code berechnet die Regressionsgerade der Standardabweichung in Abhängigkeit von der Anzahl der Steine auf dem Spielfeld mit $sklearn$ und stellt diese grafisch  dar.

In [ ]:
model = lm.LinearRegression()
model.fit(np.array(x).reshape(-1, 1), np.array(y))

plt.figure(figsize=(15, 10))
sns.set(style='whitegrid')
plt.scatter(X, y)
plt.axvline(x=0.0, c='k')
plt.axhline(y=0.0, c='k')
plt.plot(x, x * model.coef_ + model.intercept_)
plt.xlabel('number of disks on the board')
plt.ylabel('standard deviation')
plt.title('Standard Deviation')
plt.show()

Ausgeben der Funktion der Regressionsgerade, welche die Standardabweichung in Abhängigkeit von der Anzahl an Spielsteinen auf dem Spielfeld berechnet

In [ ]:
print('x *', model.coef_[0], '+', model.intercept_)

Varianz, erklärte Varianz und Standardabweichung berechnen

In [ ]:
import numpy as np
import sklearn.metrics as skl
variance = np.var(np.stack([shallow, deep], axis=1))
print(f'variance: {variance}')
explained_variance = skl.explained_variance_score(shallow, deep)
print(f'standard distribution: {np.sqrt(variance)}')
print(f'explained variance: {explained_variance}')